In [81]:
import os 
# from data_loader.mail  import  Email
mails = []
DIR = "/net/people/tutorial/tutorial014/wspole/data/mail-dataset"
for f  in os.listdir(DIR):
    m = "".join(open(DIR+"/"+f, "r").readlines())
    mails.append(m)
    
mails[0]


'\nOd: Tomasz Urban <tomasz.urban@alphacloud.com>\nWysłano: 2025-04-03 09:18\nDo: Katarzyna Pawlak <katarzyna.pawlak@szpilex.ai>\nDW: Marek Jaworski <marek.jaworski@alphacloud.com>, support-leads@alphacloud.com\nTemat: [Eskalacja] RAG Docs – błędne odpowiedzi i stara dokumentacja\n\nCześć Kasiu,\n\nmuszę podbić temat RAG-a, bo od wczoraj mamy kilka zgłoszeń od supportu:\n\n1) Case #SUP-23109:\n   - pytanie agenta: “Czy klient na planie STANDARD ma dostęp do API /v3/billing/export?”\n   - RAG odpowiedział: “Tak, endpoint dostępny od planu STANDARD wzwyż”.\n   - Faktycznie: od stycznia endpoint jest tylko w ENTERPRISE (policy change).\n\n2) Case #SUP-23127:\n   - pytanie o limit projektów w v3,\n   - RAG zwrócił fragmenty dokumentacji z v2 (max 5 projektów), podczas gdy w v3 jest 20.\n\nWygląda na to, że:\n- albo indeks zawiera stare wersje dokumentów,\n- albo filtr po “version”/“valid_to” nie działa tak, jak zakładaliśmy.\n\nNa razie poprosiłem team leadów, żeby:\n- traktowali odpowiedz

In [82]:
from pydantic import BaseModel, Field
from typing import Any, List, Optional


class Email(BaseModel):
    from_: str = Field(..., description="Sender email address")
    to: List[str] = Field(..., description="Recipient email addresses")
    topic: str = Field(..., description="Email topic or subject")
    data: str = Field(..., description="Full raw email content or body")
    summary: Optional[str] = Field(None, description="Auto-generated summary of the content")
    extra: dict[str, Any ]

    class ConfigDict:
        json_schema_extra = {
            "example": {
                "from_": "alice@example.com",
                "to": ["bob@example.com"],
                "topic": "Meeting Update",
                "data": "Hi Bob,\nThe meeting is moved to Monday at 2 PM.\nRegards,\nAlice",
                "summary": "Meeting rescheduled to Monday at 2 PM."
            }
        }


In [ ]:
from openai import OpenAI

TOKEN = os.environ["CYFRANET_TOKEN"]

client = OpenAI(
    api_key=TOKEN, 
    base_url="https://llmlab.plgrid.pl/api/v1"
)

response =  client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
     messages=[{"role": "user", "content": f"Wrtie mail addresses of recipents: {m}. Only output mail addreeses, no additional text"}],
    # input=f"Znajdz tytul maila" # {m}"
)
    
response.choices[0].message.content

In [77]:
def extract_data(client, model,  text): 
    response =  client.chat.completions.create(
    model=model,
     messages=[{"role": "user", "content": text}], # f"Wrtie mail addresses of recipents: {m}. Only output mail addreeses, no additional text"}],
    # input=f"Znajdz tytul maila" # {m}"
    )
    return response.choices[0].message.content


def process_mail(mail: str, client: OpenAI) -> Email:
    # from 
    from_ = extract_data(client, "meta-llama/Llama-3.3-70B-Instruct", f"Wrtie mail addresses of sender: {mail}. This should be one email (usually after 'OD:'). Only output one mail address, no additional text") 
    # to 
    to_ = extract_data(client, "meta-llama/Llama-3.3-70B-Instruct", f"Wrtie mail addresses of recipents: {mail}. This should be at least one email (usually after 'DO:') at the top of the content.  Only output mail addreeses, no additional text")
    to_ = to_.split("\n")
    # topic 
    topic = extract_data(client, "meta-llama/Llama-3.3-70B-Instruct", f"Wrtie mail main topic: {mail}. This should be a field called 'Temat'. Only output mail title, no additional text")
    # data
    data = mail 
    # summary 
    summary = extract_data(client, "meta-llama/Llama-3.3-70B-Instruct", f"Summarize mail content in 3 sentences : {mail}. Only output summary, no additional text")
    # etra 
    tags = extract_data(client, "meta-llama/Llama-3.3-70B-Instruct", f"Wrtie potential tags, categroies: {summary}. Only output categories separated by semicolon, no additional text")

    extra = {"tags": tags}
    e = Email(from_=from_, to=to_, topic=topic, data=data, summary=summary, extra=extra)
    return e
    
email = process_mail(mails[0], client)

In [ ]:
for m in mails:
    try:
        email = process_mail(m, client)
        print(email.topic)
    except:
        print("-----")
        print(m)
        continue